In [2]:
import pandas as pd

# Load the provided GTFS files into DataFrames
stops_df = pd.read_csv('stops.txt')
routes_df = pd.read_csv('routes.txt')
trips_df = pd.read_csv('trips.txt')
stop_times_df = pd.read_csv('stop_times.txt')
calendar_df = pd.read_csv('calendar.txt')
calendar_dates_df = pd.read_csv('calendar_dates.txt')

# Step 1: Merge stops and stop_times based on 'stop_id'
merged_stops_stop_times = pd.merge(stop_times_df, stops_df, on='stop_id')

# Step 2: Merge routes and trips based on 'route_id'
merged_routes_trips = pd.merge(routes_df, trips_df, on='route_id')

# Step 3: Merge merged_routes_trips and the result from Step 1 based on 'trip_id'
merged_routes_trips_stops = pd.merge(merged_routes_trips, merged_stops_stop_times, on='trip_id')

# Step 4: Merge calendar and calendar_dates based on 'service_id'
merged_calendar = pd.merge(calendar_df, calendar_dates_df, on='service_id', how='left')

# Step 5: Merge merged_routes_trips_stops and merged_calendar based on 'service_id'
combined_gtfs_data = pd.merge(merged_routes_trips_stops, merged_calendar, on='service_id', how='left')

# Display the combined DataFrame with all the GTFS attributes
print(combined_gtfs_data.head)


<bound method NDFrame.head of         route_id agency_id route_short_name      route_long_name  route_desc  \
0              1     SORTA                1  Mt. Adams Eden Park         NaN   
1              1     SORTA                1  Mt. Adams Eden Park         NaN   
2              1     SORTA                1  Mt. Adams Eden Park         NaN   
3              1     SORTA                1  Mt. Adams Eden Park         NaN   
4              1     SORTA                1  Mt. Adams Eden Park         NaN   
...          ...       ...              ...                  ...         ...   
669920        90     SORTA               M+  MetroPlus - Kenwood         NaN   
669921        90     SORTA               M+  MetroPlus - Kenwood         NaN   
669922        90     SORTA               M+  MetroPlus - Kenwood         NaN   
669923        90     SORTA               M+  MetroPlus - Kenwood         NaN   
669924        90     SORTA               M+  MetroPlus - Kenwood         NaN   

        r

In [ ]:
import pandas as pd
from keplergl import KeplerGl

# Assuming you have the combined_gtfs_data DataFrame containing all the GTFS data

# Filter out only the relevant columns for stops and routes
stops_data = combined_gtfs_data[['stop_id', 'stop_name', 'stop_lat', 'stop_lon', 'route_id', 'route_short_name', 'route_long_name']]

# Create a Kepler GL map with the transit schedule data
map_config = {
    "version": "v1",
    "config": {
        "mapState": {
            "latitude": 39.0,  # Replace with the center latitude of your area
            "longitude": -84.5,  # Replace with the center longitude of your area
            "zoom": 10  # Adjust the zoom level as needed
        }
    }
}

kepler_map = KeplerGl(height=600, config=map_config)

# Add data to the map with layer selector enabled
kepler_map.add_data(data=stops_data, name='Transit Stops')

# Enable layer selector to allow users to select routes
kepler_map.config = {
    "version": "v1",
    "config": {
        "mapState": {
            "latitude": 39.0,
            "longitude": -84.5,
            "zoom": 10
        },
        "visState": {
            "layers": [
                {
                    "id": "Transit Stops",
                    "type": "point",
                    "config": {
                        "dataId": "Transit Stops",
                        "label": "Transit Stops",
                        "columns": {
                            "lat": "stop_lat",
                            "lng": "stop_lon",
                            "altitude": None
                        },
                        "isVisible": True,
                        "visConfig": {
                            "radius": 10,
                            "fixedRadius": False,
                            "opacity": 0.8,
                            "outline": False,
                            "thickness": 1,
                            "strokeColor": [0, 0, 0],
                            "colorRange": {
                                "name": "ColorBrewer YlGnBu-6",
                                "type": "sequential",
                                "category": "ColorBrewer",
                                "colors": [
                                    "#006837",
                                    "#1a9850",
                                    "#66bd63",
                                    "#a6d96a",
                                    "#d9ef8b",
                                    "#ffffbf"
                                ]
                            },
                            "radiusRange": [0, 50],
                            "hi-precision": False
                        },
                        "textLabel": [
                            {
                                "field": None,
                                "color": [255, 255, 255],
                                "size": 18,
                                "offset": [0, 0],
                                "anchor": "start",
                                "alignment": "center"
                            }
                        ]
                    }
                },
                # Add a new layer for each route with their respective stops
                {
                    "id": "Route Stops",
                    "type": "point",
                    "config": {
                        "dataId": "Transit Stops",
                        "label": "Route Stops",
                        "columns": {
                            "lat": "stop_lat",
                            "lng": "stop_lon",
                            "altitude": None,
                            "route_id": "route_id"
                        },
                        "isVisible": False,
                        "visConfig": {
                            "radius": 8,
                            "fixedRadius": False,
                            "opacity": 0.8,
                            "outline": False,
                            "thickness": 1,
                            "strokeColor": [0, 0, 0],
                            "colorRange": {
                                "name": "ColorBrewer RdBu-6",
                                "type": "sequential",
                                "category": "ColorBrewer",
                                "colors": [
                                    "#fc8d59",
                                    "#ffffbf",
                                    "#91bfdb"
                                ]
                            },
                            "radiusRange": [0, 50],
                            "hi-precision": False
                        },
                        "textLabel": [
                            {
                                "field": None,
                                "color": [255, 255, 255],
                                "size": 18,
                                "offset": [0, 0],
                                "anchor": "start",
                                "alignment": "center"
                            }
                        ]
                    }
                }
            ],
            "interactionConfig": {
                "tooltip": {
                    "fieldsToShow": {
                        "Transit Stops": [
                            "route_id",
                            "stop_name",
                            "route_short_name",
                            "route_long_name"
                        ]
                    },
                    "enabled": True
                },
                "brush": {
                    "size": 0.5,
                    "enabled": False
                },
                "geocoder": {
                    "enabled": False
                },
                "coordinate": {
                    "enabled": False
                }
            },
            "layerBlending": "normal",
            "splitMaps": []
        }
    }
}

# Display the Kepler GL map with layer selector
kepler_map


In [ ]:
import pandas as pd
from keplergl import KeplerGl

# Assuming you have the combined_gtfs_data DataFrame containing all the GTFS data

# Create a Kepler GL map with the transit schedule data
map_config = {
    "version": "v1",
    "config": {
        "mapState": {
            "latitude": 39.0,  # Replace with the center latitude of your area
            "longitude": -84.5,  # Replace with the center longitude of your area
            "zoom": 10  # Adjust the zoom level as needed
        }
    }
}

kepler_map = KeplerGl(height=600, config=map_config)

# Display the Kepler GL map
kepler_map


In [ ]:

# Filter out only the relevant columns for stops and routes
stops_data = combined_gtfs_data[['stop_id', 'stop_name', 'stop_lat', 'stop_lon', 'route_id', 'route_short_name', 'route_long_name']]

# Add data to the map with layer selector enabled
kepler_map.add_data(data=stops_data, name='Transit Stops')


In [ ]:
# Filter stops data for a specific route (e.g., route_id '1')
route_1_stops_data = stops_data[stops_data['route_short_name'] == '1']
print(route_1_stops_data)
# Add data to the map with layer selector enabled
kepler_map.add_data(data=stops_data, name='Transit Stops')

# Add a new layer for route 1 stops
kepler_map.add_data(data=route_1_stops_data, name='Route 1 Stops')


In [ ]:
import pandas as pd
from keplergl import KeplerGl

# Assuming you have the combined_gtfs_data DataFrame containing all the GTFS data

# Step 1: Preprocess the data to get the frequency of buses at each stop
stop_frequency = combined_gtfs_data['stop_id'].value_counts().reset_index()
stop_frequency.columns = ['stop_id', 'frequency']

# Step 2: Merge stop frequencies with stop locations
stops_data_with_frequency = pd.merge(combined_gtfs_data[['stop_id', 'stop_name', 'stop_lat', 'stop_lon', 'route_id']],
                                     stop_frequency, on='stop_id')

# Step 3: Create a Kepler GL map and add the data as layers
kepler_map = KeplerGl(height=600)

# Layer 1: Heatmap to show the density of stops based on frequency
kepler_map.add_data(data=stops_data_with_frequency, name='Stop Frequency Heatmap')

# Layer 2: Create separate layers for each route's trips
unique_routes = combined_gtfs_data['route_id'].unique()

for route_id in unique_routes:
    route_trips = combined_gtfs_data[combined_gtfs_data['route_id'] == route_id][['stop_lat', 'stop_lon']]
    kepler_map.add_data(data=route_trips, name=f'Route {route_id} Trips')

# Layer 3: Stops to display all the stops
kepler_map.add_data(data=combined_gtfs_data, name='All Stops')

# Create a weighted heatmap layer using the stop frequencies
heatmap_layer = {
    "id": "Weighted Heatmap",
    "type": "heatmap",
    "config": {
        "dataId": "Stop Frequency Heatmap",
        "label": "Weighted Heatmap",
        "color": [255, 0, 0],
        "columns": {
            "lat": "stop_lat",
            "lng": "stop_lon",
            "weight": "frequency"
        },
        "isVisible": True,
        "visConfig": {
            "opacity": 0.7,
            "colorRange": {
                "name": "ColorBrewer RdBu-10",
                "type": "diverging",
                "category": "ColorBrewer",
                "colors": [
                    "#67001f",
                    "#b2182b",
                    "#d6604d",
                    "#f4a582",
                    "#fddbc7",
                    "#f7f7f7",
                    "#d1e5f0",
                    "#92c5de",
                    "#4393c3",
                    "#2166ac",
                    "#053061"
                ]
            },
            "radius": 50
        }
    }
}

kepler_map.config = {
    "version": "v1",
    "config": {
        "mapState": {
            "latitude": combined_gtfs_data['stop_lat'].mean(),
            "longitude": combined_gtfs_data['stop_lon'].mean(),
            "zoom": 10
        },
        "mapStyle": {
            "styleType": "dark"
        },
        "visState": {
            "layers": [heatmap_layer]
        }
    }
}

# Display the Kepler GL map
kepler_map


In [ ]:
# Calculate stop frequencies
stop_frequencies = combined_gtfs_data['stop_id'].value_counts()


In [ ]:
# Define the number of top stops to select
top_n_stops = 100

# Get the stop IDs of the top N stops
top_stops = stop_frequencies.head(top_n_stops).index

# Filter the combined GTFS data to include only the top stops
selected_stops_data = combined_gtfs_data[combined_gtfs_data['stop_id'].isin(top_stops)]


In [ ]:
from keplergl import KeplerGl

# Create a KeplerGL map
kepler_map = KeplerGl(height=600)

# Add data to the map
kepler_map.add_data(data=selected_stops_data, name='Highest Frequency Stops')

# Display the map
kepler_map


In [ ]:
import networkx as nx

# Create an empty directed graph
G = nx.DiGraph()

# Add edges to the graph based on stop sequences in the GTFS data
for _, trip_data in combined_gtfs_data.groupby('trip_id'):
    stops = trip_data['stop_id'].tolist()
    for i in range(len(stops) - 1):
        G.add_edge(stops[i], stops[i + 1])


In [ ]:
# Calculate degree centrality
degree_centrality = nx.degree_centrality(G)

# Calculate betweenness centrality
betweenness_centrality = nx.betweenness_centrality(G)

# Calculate closeness centrality
closeness_centrality = nx.closeness_centrality(G)


In [ ]:
# Define the number of top stops to select
top_n_stops = 100

# Get the stop IDs of the top N stops for each centrality measure
top_degree_stops = sorted(degree_centrality, key=degree_centrality.get, reverse=True)[:top_n_stops]
top_betweenness_stops = sorted(betweenness_centrality, key=betweenness_centrality.get, reverse=True)[:top_n_stops]
top_closeness_stops = sorted(closeness_centrality, key=closeness_centrality.get, reverse=True)[:top_n_stops]

# Combine the top stops from all centrality measures
selected_stops = set(top_degree_stops + top_betweenness_stops + top_closeness_stops)

# Filter the combined GTFS data to include only the selected stops
selected_stops_data = combined_gtfs_data[combined_gtfs_data['stop_id'].isin(selected_stops)]


In [ ]:
# Create a KeplerGL map with the map configuration
map_config = {
    "version": "v1",
    "config": {
        "mapState": {
            "latitude": 39.0,  # Replace with the center latitude of your area
            "longitude": -84.5,  # Replace with the center longitude of your area
            "zoom": 10  # Adjust the zoom level as needed
        }
    }
}

# Create a KeplerGL map
kepler_map = KeplerGl(height=600, config=map_config)


# Add data to the map
kepler_map.add_data(data=selected_stops_data, name='Centrality-based Stops')

# Display the map
kepler_map
